# 08_02: Summarizing distributions

In [ ]:
import math
import collections
import dataclasses
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as pp

In [ ]:
incomes = pd.read_csv('incomes.csv').set_index(['country', 'year'])
incomes

In [ ]:
incomes.groupby(['country','year']).count()

In [ ]:
incomes.loc['USA', 2015].min()

In [ ]:
incomes.loc['USA', 2015].max()

In [ ]:
incomes.loc['USA', 2015].mean()

In [ ]:
incomes.groupby(['country', 'year']).yearly.mean()

In [ ]:
incomes.groupby(['country', 'year']).var()

In [ ]:
incomes.groupby(['country', 'year']).std()

In [ ]:
pd.DataFrame({'25%': incomes.groupby(['country', 'year']).yearly.quantile(0.25),
              '50%': incomes.groupby(['country', 'year']).yearly.median(),
              '75%': incomes.groupby(['country', 'year']).yearly.quantile(0.75)})

In [ ]:
import scipy.stats

In [ ]:
incomes.groupby(['country', 'year']).yearly.apply(lambda d: scipy.stats.percentileofscore(d, 5000))

In [ ]:
incomes.groupby(['year', 'country']).yearly.describe() / incomes.loc['USA', 2015].yearly.describe()